In [1]:
library(disk.frame)

Loading required package: dplyr

Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union

Loading required package: purrr
Registered S3 method overwritten by 'pryr':
  method      from
  print.bytes Rcpp

## Message from disk.frame:
We have 1 workers to use with disk.frame.
To change that, use setup_disk.frame(workers = n) or just setup_disk.frame() to use the defaults.


It is recommended that you run the following immediately to set up disk.frame with multiple workers in order to parallelize your operations:


```r
# this will set up disk.frame with multiple workers
setup_disk.frame()
# this will allow unlimited amount of data to be passed from worker to worker
options(future.globals.maxSize = Inf)
```




Attaching package: ‘disk.frame’

The following objects are masked from ‘package:purrr’:

    imap, imap_dfr, map, map2

The following objects 

In [2]:
setup_disk.frame()

The number of workers available for disk.frame is 6


In [3]:
system.time(a <- csv_to_disk.frame("fm-data/perf/Performance_2000Q1.txt"))

   user  system elapsed 
 21.058   1.492   8.710 